In [ ]:
#this is 1d CNN with CONV1D+ temporal pooling

In [ ]:
#Input (2 × 18)
# → Conv1D (kernel=3)
# → BatchNorm
# → ReLU
# → Conv1D (kernel=3)
# → BatchNorm
# → ReLU
# → Adaptive Pooling
# → Shared Representation
#    ├── Class Head (Softmax)
#    └── Anomaly Head (Sigmoid)


In [32]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


In [53]:
FEATURES = [
    "Tot Fwd Pkts",
    "Tot Bwd Pkts",
    "Flow Pkts/s",
    "Flow Byts/s",
    "Flow Duration",
    "Flow IAT Mean",
    "Flow IAT Std",
    "Fwd IAT Mean",
    "Bwd IAT Mean",
    "Pkt Len Mean",
    "Pkt Len Std",
    "Pkt Len Var",
    "SYN Flag Cnt",
    "ACK Flag Cnt",
    "RST Flag Cnt",
    "FIN Flag Cnt",
    "Down/Up Ratio",
    "Init Fwd Win Byts"
]

Label_MAP = {
    "Normal": 0,
    "DDOS":1
}


In [219]:
#Dataset class (Preprocessing logic)
window_size=10
class SDNIDSDataset(Dataset):
    def __init__(self, csv_path, window_size=10, time_steps=2):
        self.df = pd.read_csv(csv_path)
        self.window_size = window_size
        self.time_steps = time_steps

        self._clean_dataframe()
        self.samples, self.Labels = self._build_samples()

    def _clean_dataframe(self):
        self.df.columns = [c.strip() for c in self.df.columns]

        for feat in FEATURES:
            if feat not in self.df.columns:
                self.df[feat] = 0.0

        self.df = self.df.dropna(subset=["Label"])
        self.df["Label"] = self.df["Label"].astype(int)   #.map(Label_MAP)
        self.df = self.df.dropna(subset=["Label"])
        self.df["Label"] = self.df["Label"].astype(int)

        self.df[FEATURES] = self.df[FEATURES].replace([np.inf, -np.inf], 0)
        self.df[FEATURES] = self.df[FEATURES].fillna(0)
        
        self.df = self.df.sample(frac=1.0, random_state=42).reset_index(drop=True)


    def _aggregate_window(self, window_df):
        return window_df[FEATURES].mean().values.astype(np.float32)

#code i adapter after having all the window as an attack
    def _build_samples(self):
        windows = []
        window_Labels = []
    
        # Sliding window with stride = 1
        for i in range(0, len(self.df) - self.window_size + 1):
            window_df = self.df.iloc[i:i + self.window_size]
    
            feat_vec = self._aggregate_window(window_df)
    
            # # Use majority vote instead of OR-label
            # Label = 1 if window_df["Label"].mean() > 0.5 else 0
            # OR threshold
            threshold = 2
            Label = 1 if window_df["Label"].sum() >= threshold else 0
    
            if i < 5:  # debug
                print("Window Labels:", window_df["Label"].tolist(), "->", Label)
    
            windows.append(feat_vec)
            window_Labels.append(Label)
    
        samples = []
        Labels = []
    
        for i in range(self.time_steps - 1, len(windows)):
            stacked = np.stack(
                windows[i - self.time_steps + 1:i + 1],
                axis=0
            )
            samples.append(stacked)
            Labels.append(window_Labels[i])
    
        return (
            torch.from_numpy(np.array(samples, dtype=np.float32)),
            torch.from_numpy(np.array(Labels, dtype=np.int64))
        )
   
# #original code wwe wrote at the start
#     def _build_samples(self):
#         windows = []
#         window_Labels = []
    
#         # IMPORTANT: sort by time
#         if "Timestamp" in self.df.columns:
#             self.df = self.df.sort_values("Timestamp").reset_index(drop=True)
    
#         # Sliding window with stride = 1
#         for i in range(0, len(self.df) - self.window_size + 1):
#             window_df = self.df.iloc[i:i + self.window_size]
    
#             feat_vec = self._aggregate_window(window_df)
#             Label = window_df["Label"].max()  # IDS OR-Labeling
    
#             if i < 5:  # debug
#                 print("Window Labels:", window_df["Label"].tolist(), "->", Label)
    
#             windows.append(feat_vec)
#             window_Labels.append(Label)
    
#         samples = []
#         Labels = []
    
#         for i in range(self.time_steps - 1, len(windows)):
#             stacked = np.stack(
#                 windows[i - self.time_steps + 1:i + 1],
#                 axis=0
#             )
#             samples.append(stacked)
#             Labels.append(window_Labels[i])
    
#         return (
#             torch.from_numpy(np.array(samples, dtype=np.float32)),
#             torch.from_numpy(np.array(Labels, dtype=np.int64))
#         )


    def __len__(self):
        return len(self.Labels)

    def __getitem__(self, idx):
        return self.samples[idx], self.Labels[idx]


In [220]:
#test the csv file here

df = pd.read_csv("ICMP_Flood_Attack.csv")
print(df.columns.tolist())
print(df.head())
print(df.shape)
print(df["Label"].value_counts())


['disp_pakt', 'rate_pkt_in', 'disp_interval', 'mean_byte', 'gfe', 'g_usip', 'rfip', 'avg_flow_dst', 'Label', 'switch_id', 'time', 'mean_pkt', 'disp_byte', 'avg_durat', 'gsp']
   disp_pakt  rate_pkt_in  disp_interval  mean_byte  gfe  g_usip  rfip  \
0   0.000000  2151.000000   4.810000e+17        0.0    0       0   0.0   
1   0.000000   493.600000   1.800000e+17        0.0   17       8   0.0   
2  78.342760   518.333333   2.060000e+17    18417.0    0       0   1.0   
3   1.549193   464.000000   3.090000e+17       98.0    0       0   1.0   
4   1.549193   578.333333   1.610000e+17       98.0    0       0   1.0   

   avg_flow_dst  Label  switch_id                  time   mean_pkt  \
0             3      0          2  11/16/2022, 12:17:10   0.000000   
1           282      1          6  11/16/2022, 09:55:34   0.000000   
2            84      0          2  02/06/2023, 21:25:06  12.380952   
3             6      0          3  02/21/2023, 08:24:06   1.000000   
4             6      0        

In [221]:
df["Label"] = df["Label"].map(Label_MAP)
print(df["Label"].value_counts())

Series([], Name: count, dtype: int64)


In [222]:
#Creating dataset here
dataset = SDNIDSDataset(
    csv_path="ICMP_Flood_Attack.csv",
    # window_size=10,   # ~1 second worth of flows
    time_steps=2
)
df = pd.read_csv("ICMP_Flood_Attack.csv")
print(df["Label"].value_counts())
print("Total samples:", len(dataset))

Labels = dataset.Labels
print("Unique Labels:", torch.unique(Labels))
print("Counts:", torch.bincount(Labels))
print("Sample shape:", dataset[0][0].shape)

Window Labels: [0, 0, 0, 0, 1, 1, 0, 0, 0, 0] -> 1
Window Labels: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0] -> 1
Window Labels: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0] -> 1
Window Labels: [0, 1, 1, 0, 0, 0, 0, 0, 0, 0] -> 1
Window Labels: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0] -> 1
Label
0    18457
1     9866
Name: count, dtype: int64
Total samples: 28313
Unique Labels: tensor([0, 1])
Counts: tensor([ 2528, 25785])
Sample shape: torch.Size([2, 18])


In [225]:
#testing if dataset is loaded properly and Labelled as well
print(np.unique(dataset.Labels.numpy(), return_counts=True))
print(df["Label"].unique())


(array([0, 1]), array([ 2528, 25785]))
[0 1]


In [226]:
Labels = dataset.Labels.numpy()
indices = np.arange(len(dataset))

print("Overall Label distribution:", np.unique(Labels, return_counts=True))


Overall Label distribution: (array([0, 1]), array([ 2528, 25785]))


In [227]:
#CNN MODAL BELOW

In [228]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SDN_CNN(nn.Module):
    def __init__(self, num_features=18, num_classes=2):
        super().__init__()

        # Conv blocks
        self.conv1 = nn.Conv1d(
            in_channels=num_features,
            out_channels=32,
            kernel_size=2,
            padding=0
        )
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(
            in_channels=32,
            out_channels=64,
            kernel_size=1
        )
        self.bn2 = nn.BatchNorm1d(64)

        # Global pooling
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        # Heads
        self.classifier = nn.Linear(64, num_classes)
        self.anomaly_head = nn.Linear(64, 1)  # optional, future use

    def forward(self, x):
        """
        x: (B, T, F)
        """
        # Rearrange for Conv1D
        x = x.permute(0, 2, 1)  # (B, F, T)

        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))

        x = self.global_pool(x).squeeze(-1)  # (B, 64)

        class_logits = self.classifier(x)
        anomaly_score = torch.sigmoid(self.anomaly_head(x))

        return class_logits, anomaly_score

#testing        
#print("hi")

In [229]:
#Sanity check of CNN modal up
model = SDN_CNN(num_features=18, num_classes=2)
model.eval()  # IMPORTANT

x, y = dataset[0]
x = x.unsqueeze(0)

with torch.no_grad():
    logits, anomaly = model(x)

print("Logits shape:", logits.shape)
print("Anomaly score:", anomaly.item())



Logits shape: torch.Size([1, 2])
Anomaly score: 0.49741655588150024


In [230]:
train_dataset = Subset(dataset, train_idx)
val_dataset   = Subset(dataset, val_idx)

In [231]:
#splitiing the dataset
Labels = dataset.Labels.numpy()
indices = np.arange(len(dataset))

train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,
    stratify=dataset.Labels,
    random_state=42
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)


In [232]:
#verifiy Spliting
train_Labels = Labels[train_idx]
val_Labels   = Labels[val_idx]

print("Train Labels:", np.unique(train_Labels, return_counts=True))
print("Val Labels:", np.unique(val_Labels, return_counts=True))


Train Labels: (array([0, 1]), array([ 2022, 20628]))
Val Labels: (array([0, 1]), array([ 506, 5157]))


In [233]:
#training modal now
#DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)


In [234]:
#check if cuda is being used of not
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [256]:
#CLASS WEIGHTS
class_counts = np.bincount(train_Labels)

weights = class_counts.sum() / (2 * class_counts)
# class_weights = torch.tensor(
#     class_weights,
#     dtype=torch.float32
# ).to(device)
class_weights = weights.detach().clone().to(device)
print("Class counts:", class_counts)
print("Class weights:", class_weights)
print("Type:", type(class_weights))



AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [238]:
#Model, Loss, Optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)

model = SDN_CNN(
    num_features=18,
    num_classes=2
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



In [239]:
#training function fo one as to keep gpu safe and easier to see whats happening
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()

    total_loss = 0.0
    all_preds = []
    all_targets = []

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits, _ = model(X)
        loss = criterion(logits, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

    avg_loss = total_loss / len(loader)
    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)

    return avg_loss, acc, f1



In [240]:
#validation loop
def evaluate(model, loader, device):
    model.eval()

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            logits, _ = model(X)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(y.cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)

    return acc, f1



In [241]:
threshold = 2
window_labels = []
for i in range(len(df) - window_size + 1):
    win = df.iloc[i:i+window_size]
    label = 1 if win["Label"].sum() >= threshold else 0
    window_labels.append(label)

unique, counts = np.unique(window_labels, return_counts=True)
print("Window label distribution:", dict(zip(unique, counts)))


Window label distribution: {np.int64(0): np.int64(14694), np.int64(1): np.int64(13620)}


In [242]:
#actual final training
num_epochs = 30

for epoch in range(num_epochs):
    train_loss, train_acc, train_f1 = train_one_epoch(
        model,
        train_loader,
        optimizer,
        criterion,
        device
    )

    val_acc, val_f1 = evaluate(
        model,
        val_loader,
        device
    )

    print(
        f"Epoch {epoch+1:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Train F1: {train_f1:.4f} | "
        f"Val Acc: {val_acc:.4f} | "
        f"Val F1: {val_f1:.4f}"
    )


Epoch 01 | Train Loss: 0.6905 | Train Acc: 0.9097 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 02 | Train Loss: 0.6904 | Train Acc: 0.9097 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 03 | Train Loss: 0.6906 | Train Acc: 0.9096 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 04 | Train Loss: 0.6903 | Train Acc: 0.9096 | Train F1: 0.9526 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 05 | Train Loss: 0.6911 | Train Acc: 0.9097 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 06 | Train Loss: 0.6907 | Train Acc: 0.9097 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 07 | Train Loss: 0.6905 | Train Acc: 0.9096 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 08 | Train Loss: 0.6906 | Train Acc: 0.9097 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 09 | Train Loss: 0.6908 | Train Acc: 0.9097 | Train F1: 0.9527 | Val Acc: 0.9149 | Val F1: 0.9556
Epoch 10 | Train Loss: 0.6900 | Train Acc: 0.9097 | Train F1: 0.

In [252]:
out = model(x)
print(type(out), len(out))


<class 'tuple'> 2


In [254]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        y = y.to(device)

        logits,_ = model(x)
        probs  = torch.sigmoid(logits)
        preds  = (probs >= 0.5).long()

        all_preds.append(preds.cpu())
        all_labels.append(y.cpu())

all_preds  = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

print("Pred counts:", np.unique(all_preds, return_counts=True))
print("True counts:", np.unique(all_labels, return_counts=True))


Pred counts: (array([0]), array([11326]))
True counts: (array([0, 1]), array([ 482, 5181]))


In [250]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(all_labels, all_preds))


ValueError: Found empty input array (e.g., `y_true` or `y_pred`) while a minimum of 1 sample is required.